In [1]:
import os
import torch
import json
from io import BytesIO
import numpy as np
import torch.nn as nn
from tqdm import tqdm, trange
from main import GNNModel, CustomDataset
import matplotlib.pyplot as plt
import imageio.v2 as imageio

In [2]:
device = torch.device('cpu')

In [ ]:
data_path = '/Users/reza/Career/DMLab/SURROGATE/Data/trash/train'
result_path = '/Users/reza/Career/DMLab/SURROGATE/results/laplace/gnn'
cfg_path = os.path.join(result_path, 'cfg.json')
with open(cfg_path, 'r') as f:
    cfg = json.load(f)
subdir_paths = sorted(os.listdir(data_path))
cr_paths = [os.path.join(data_path, p, 'br.hdf') for p in subdir_paths]

In [21]:
starting_slice = 0

dataset = CustomDataset(
    cr_paths,
    100
)

In [22]:
model = GNNModel(1, int(cfg['hidden_dim']), 1).to(device)

In [23]:
state_path = os.path.join(result_path, '99.pth')
model.load_state_dict(torch.load(state_path, map_location='cpu'))

/var/folders/bv/7h0f0hns2y72yqgg3ww8qkkm0000gn/T/ipykernel_1598/1569827207.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(state_path, m

<All keys matched successfully>

# Validation partition

In [24]:
i = 1
slices, sim_cube = dataset[i]
with torch.no_grad():
    yhat = []
    x = slices[0]
    for _ in range(len(slices) - 1):
        yhat_k = model(x.to(device))
        yhat.append(yhat_k.reshape(101, 101))
        x = dataset.get_graph_data(yhat_k)

In [25]:
yhat = torch.stack(yhat, dim=0)
y = sim_cube

In [26]:
yhat.shape, y.shape

(torch.Size([99, 101, 101]), torch.Size([99, 101, 101]))

In [27]:
frames = []
for step in trange(yhat.shape[0]):
    val = y[step, :, :]
    pred = yhat[step, :, :]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    cmap = "jet_r"

    # # Plot first subplot
    im1 = axes[0].imshow(val, cmap=cmap)
    axes[0].set_title(f"Actual Slice (k={step+1+starting_slice})")

    # Plot second subplot
    im2 = axes[1].imshow(pred, cmap=cmap)
    axes[1].set_title(f"Predicted Slice (k={step+1+starting_slice})")

    # Create a single colorbar
    cbar = fig.colorbar(im1, ax=axes, orientation="vertical", fraction=0.05, pad=0.02)

    # plt.tight_layout()
    # plt.show()

    # plt.show()
    buf = BytesIO()
    plt.savefig(buf, format='png')
    frames.append(imageio.imread(buf))
    plt.close()

100%|██████████| 99/99 [00:10<00:00,  9.47it/s]


In [28]:
output_filename = f"br_gnn.mp4"
fps = 10  # Adjust frames per second as needed

# Create video directly from frames in memory
with imageio.get_writer(output_filename, fps=fps) as writer:
    for frame in frames:
        writer.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1200, 600) to (1200, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
